In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/Data Mining"

/content/drive/My Drive/Data Mining


In [3]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [22]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.metrics.pairwise import linear_kernel
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

from tqdm.notebook import tqdm

import os

In [129]:
# df_data = pd.read_csv("../Dataset/processedDataWithThaliaAdded.csv", nrows=10000,delimiter='|')
df_data = pd.read_csv("Data/processedDataWithThaliaAddedWithTranslatedTopics.csv")
df_data1 = pd.read_csv('Data/processedDataForRemainedPart.csv', delimiter='|')
df_data_final = pd.concat([df_data, df_data1], ignore_index=True)
df_data_final

,Unnamed: 0,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,isbn,publisher_raw,publisher_api,publishedDate,description,subtopics,thumbnail,maturityRating,pageCount,isEbook,main topic
0,0.0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",tripion minaccia dallo spazio,Stefano Grimaldi,Stefano Grimaldi,stefano grimaldi,it,Italienisch,2.0,9.781292e+12,Lulu.com,Lulu.com,2014-07-06,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,,http://books.google.com/books/content?id=HuPeB...,NOT_MATURE,58.0,0.0,Science fiction
1,1.0,32836,#famous,#famous,famous,Jilly Gagnon,Jilly Gagnon,"jilly gagnon, todd strasser",de,", Englisch",2.0,9.783523e+12,HarperCollins Publishers Inc,Planet!,2018-02-13,Eine turbulente Social-Media-Liebesgeschichte ...,YXQ,http://books.google.com/books/content?id=kDM5D...,NOT_MATURE,368.0,1.0,Children’s / Teenage fiction: Relationship sto...
2,2.0,38332,''Grounded'' at Fort Irwin,''Grounded'' at Fort Irwin,grounded at fort irwin,Keon Fleurima,Keon Fleurima,keon fleurima,un,Englisch,2.0,9.781477e+12,Xlibris,Xlibris Corporation,2011-11-15,Grounded is about me getting in trouble. After...,YFB,http://books.google.com/books/content?id=wY-DA...,NOT_MATURE,NaN,0.0,Children’s / Teenage general interest: Boys an...
3,3.0,51507,''It's Not Ok'',''It's Not OK'',its not ok,Amy Castle,Amy Castle,amy castle,en,Englisch,2.0,9.781493e+12,Xlibris,Xlibris Corporation,2013-11-11,"Although it's been around forever, bullying ha...",YXW,http://books.google.com/books/content?id=B8CDA...,NOT_MATURE,32.0,1.0,Children’s / Teenage fiction: General fiction
4,4.0,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,2.0,NaN,HarperCollins Publishers,NaN,2008,NaN,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,0.0,Children’s / Teenage fiction: Relationship sto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70287,NaN,71124,Été,Été,Été,Amy Culliford,Amy Culliford,NaN,un,fr,NaN,9.781427e+12,CRABTREE ROOTS LES RACINES,Les Saisons Dans Une Année,2021,Early readers are introduced to the summer sea...,YBLJ,http://books.google.com/books/content?id=hvw5z...,NOT_MATURE,16.0,0.0,YPCA21
70288,NaN,8784,Één met de dood: De Kurtherian Gambit serie Bo...,No items from Google API,Één met de dood,Michael Anderle,NaN,NaN,NaN,nl,NaN,NaN,LIGHTNING SOURCE INC,NaN,NaN,NaN,NaN,https://assets.thalia.media/img/artikel/c6b145...,NaN,NaN,NaN,FLU
70289,NaN,65034,Übernahme,Übernahme,Übernahme,Funda R. Özbay,Funda R. Özbay,NaN,un,de,NaN,9.783842e+12,Books on Demand,BoD – Books on Demand,2012,in einer zeit des krieges zwischen den usa und...,NaN,http://books.google.com/books/content?id=CrnVE...,NOT_MATURE,88.0,0.0,FL
70290,NaN,5665,Übungsheft Handlettering: Übungsbuch Für Hand ...,Übungsheft Handlettering: Übungsbuch Für Hand ...,NaN,"Sunshine Handlettering, Handlettering Heft In ...","Handlettering Heft In Din A5, Sunshine Handlet...",NaN,un,NaN,NaN,9.781093e+12,INDEPENDENTLY PUBLISHED,NaN,2019-04-03,Das erwartet Dich bei diesem Übungsheft: Blank...,NaN,http://books.google.com/books/content?id=o03vw...,NOT_MATURE,112.0,0.0,YNV


In [130]:
evaluation = pd.read_csv('evaluation.csv')
evaluation

,itemID
0,12
1,45274
2,10104
3,41371
4,14015
...,...
995,78384
996,68157
997,44964
998,12971


In [131]:
df_dataset = df_data_final[['itemID','title_thalia','author_thalia','publisher_api','main topic']]

#filter out nan values with em
df_dataset.fillna("",inplace=True)

#concat all relevant columns into one metadata column. ATM => title, author, publisher, main topic // description + keywords missing
df_dataset['Metadata'] = df_dataset[['title_thalia','author_thalia','publisher_api','main topic']].apply(lambda x: ' '.join(x), axis = 1)

#define final columns
df_dataset = df_dataset[['itemID','title_thalia','Metadata']]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [132]:
#final columns inspection
df_dataset.head()

,itemID,title_thalia,Metadata
0,26966,tripion minaccia dallo spazio,tripion minaccia dallo spazio stefano grimal...
1,32836,famous,"famous jilly gagnon, todd strasser Planet! Chi..."
2,38332,grounded at fort irwin,grounded at fort irwin keon fleurima Xlibris C...
3,51507,its not ok,its not ok amy castle Xlibris Corporation Chil...
4,13238,then he ate my boy entrancers,then he ate my boy entrancers louise rennison...


In [ ]:
indices = pd.Series(df_dataset.index,index = df_dataset["itemID"]).drop_duplicates()

In [133]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#defining stopwords
tfidf = TfidfVectorizer(stop_words='english')

#define which columns to vectorize
tfidf_matrix5 = tfidf.fit_transform(df_dataset['Metadata'])

#create new DF with values
# tfidf_df5 = pd.DataFrame(tfidf_matrix5.toarray(), index=df_dataset.index.tolist())


In [202]:
def get_recommendations(bookid, cosine_sim):
  #Get the index of the movie that matches the title
  idx = index = indices.filter(regex = f'^{bookid}$|^{bookid}\||\|{bookid}$', axis=0)
  if len(idx) == 0:
    return {'itemID': bookid, 'recommendedBook': None}
  else:
    idx = idx[0]
    # Get the pairwise similarity scores of all books with that books
    cosine_similarities = linear_kernel(tfidf_matrix5[idx:idx+1], tfidf_matrix5).flatten()
    related_books_indices = cosine_similarities.argsort()[:-7:-1]
    #Get relevant books
    books = [df_dataset.filter([i], axis=0)['itemID'].tolist()[0] for i in related_books_indices if i != idx]

    return {'itemID': bookid, 'recommendedBook':  books}

In [203]:
recommendations = {}
for index, row in evaluation.iterrows():
  book_search = row['itemID']
  recommendations[index] = get_recommendations(book_search, tfidf_matrix5)
recommendation_df = pd.DataFrame.from_dict(recommendations, orient='index')
# print('Content-based RecSyS based on Metadata')
# get_recommendations(book_search,content_correlation5, 'Similarity')

In [204]:
recommendation_df

,itemID,recommendedBook
0,12,"[32729, 53663, 6667, 2390, 39256]"
1,45274,"[7262, 15116, 6210|18662, 65147, 54336]"
2,10104,"[10143, 58103, 10317, 72922, 50866]"
3,41371,"[5376, 46995, 28401, 46077, 18617]"
4,14015,"[20173, 59077, 13103, 40294, 54587]"
...,...,...
995,78384,"[56354, 11695, 42539, 12464, 74588]"
996,68157,"[66167, 41398, 6253, 18891, 43821]"
997,44964,None
998,12971,"[56107, 26291, 49248, 35540, 15800]"


In [206]:
recommendation_df.to_csv('MetadataRecommendation.csv')

In [208]:
len(recommendation_df[recommendation_df.recommendedBook.isnull()])

60